# QBS 108 2023 Spring Project 3

## Question 1 Feed-forward Neural Network for Binary Classification 
You are given a real-world dataset to predict the prognosis of Breast Cancer patients using genetic biomarkers. Your task is to predict the breast cancer prognostic status, whose variable name is "outcome". A value of 1 is worse prognosis and a value of 0 is better prognosis. The codebook of this dataset is provided in hw3\_codebook.xlsx. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

dt_dev = pd.read_csv('BRCA-prognosis-train.csv')
all_columns = dt_dev.columns.tolist()
mutation_columns = all_columns[489:-1] #Gene mutation
expression_columns = all_columns[:489]  #Gene expression

# fix missing data
for var in mutation_columns:
    dt_dev.loc[:,var] = dt_dev[var].apply(lambda x: 0 if x == '0' else 1)
    
for var in dt_dev.columns.tolist():
    dt_dev.loc[:,var].fillna((dt_dev[var].mean()), inplace=True)

### 1.1 Data preprocessing and exploratory data analysis

In [ ]:
# your code here


### 1.2 Proposing a deep learning-based model development and evaluation plan 

### 1.3 Model Training
If you're uncertain about how to complete this section, you can refer to this blog post, https://towardsdatascience.com/pytorch-tabular-regression-428e9c9ac93. 

You can modify the code provided below to complete this question. Feel free to play around with and use different activation functions (https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity), dropout layers (https://pytorch.org/docs/stable/nn.html#dropout-layers), the number of hidden neurons, and the number of layers.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class MLP(nn.Module):
    def __init__(self, input_dim,hidden_dim,output_dim):
        super(MLP, self).__init__()
        self.fc = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                nn.ReLU(inplace=True), 
                                nn.Linear(hidden_dim, hidden_dim),
                                nn.ReLU(inplace=True),
                                nn.Linear(hidden_dim,output_dim)) 

    def forward(self, inputs):
        return self.fc(inputs)

You can use the code below to load data in a format compatible with PyTorch:

In [ ]:
class PandasDataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None):
        super().__init__()
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
        
    def __getitem__(self, idx):
        xi = torch.tensor(self.X.iloc[idx].to_numpy()).float()
        if self.y is None:
            return xi
        yi = torch.tensor(self.y.iloc[idx].item()).float()
        return xi, yi

dt_train, dt_val = train_test_split(dt_dev, test_size=0.33, random_state=42, stratify=dt_dev.outcome)
outcome = 'outcome'
X_train = dt_train.drop([outcome],1)
y_train = dt_train[outcome]
X_val = dt_val.drop([outcome],1)
y_val = dt_val[outcome]

**(a) Implement the feed-forward neural network model that you proposed using the Training Portion of the Labeled Data that you have split. Tune hyperparameters and monitor the model performance (including loss and AUC) in the validation dataset.**

In [ ]:
# your code here
batch_size = None

train_data = PandasDataset(X_train,y_train)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4)

val_data = PandasDataset(X_val,y_val)
val_loader = DataLoader(val_data, batch_size=16, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train(model, train_loader, optimizer, criterion, **kwargs):
  model.train()
  running_loss = 0.0

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  epoch_loss = running_loss / len(train_loader.dataset)
  return epoch_loss


def test(model, test_loader, criterion):
  model.eval()
  # TODO: Complete the following code to return the loss and AUC in the validation set
  """
  1. Iterate through the validation set
  2. For each batch, perform the following steps:
    a. Obtain the model output
    b. Compute and update the loss
    c. Compute and update the AUC
  7. return the loss and AUC
  """
  val_auc = None
  val_loss = None
  return val_auc, val_loss

In [ ]:
input_dim = None
hidden_dim = None
output_dim = None
num_epochs = None
lr = None
model = MLP(input_dim,hidden_dim,output_dim)


train_losses = []
val_losses = []
criterion = None
optimizer = None
best_model = None  # You don't need to replace 'None' here
best_loss = 10000
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion, epoch) # print every 2 batch
    train_losses.append(train_loss)
    val_loss = test(model, device, val_loader, criterion)
    val_losses.append(val_loss)
    if val_loss < best_loss:
      best_loss = val_loss
      best_model = model

**(b) With your selected model from the previous question, show a Confusion Matrix in the validation portion of the labeled dataset. Also report the evaluation metrics in the validation dataset, including: precision, recall, accuracy, F1 score, and AUC.**

In [ ]:
# your code here
def predict(model, data_loader):
  model.eval()
  total_labels = []
  total_predictions = []
  total_scores = []  # Probability

  # TODO: Complete the following code to return the loss and AUC in the validation set
  """
  1. Iterate through the dataset.
  2. For each batch, perform the following steps:
    a. Append the labels to the total_labels list.
    b. Obtain the model output.
    c. Compute the probability for each class, typically using the Sigmoid or Softmax function.
    d. Compute the model prediction based on the highest probability output.
    e. Append the corresponding results to the total_predictions and total_scores lists.
  3. Return the loss and AUC.
  """

  return total_labels, total_predictions, total_scores


After completing the function above, you should be able to use your best model to obtain the inputs required for the functions in sklearn.metrics. You can find all the metrics you needed at the following link: https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
#Your code here

### 1.4 Performance Evaluation

Apply your trained model to the Unlabeled Data (**BRCA-prognosis-test.csv**). Save your predicted **PROBABILITIES** in a new csv file named **`<YourNetID>`_predictions.csv**. Please submit this CSV file along with other files on Canvas.

In [ ]:
# your code here

##  Question 2 t-SNE for Data Visualization

### 2.1 t-SNE Implementation
You may find some useful content on the scikit-learn documentation page for the t-SNE algorithm, which includes an implementation in Python and examples for usage. You can find more information here: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

**(a)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# your code here

**(b)**

In [ ]:
# Your code here

* Remember to describe your findings

## Question 3 CNN for Fish Species Classification
You might find some useful information at https://pytorch.org/vision/main/models.html and at https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html.

In [ ]:
import os
import torch
from torch import nn
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Replace 'None' with a transforms.Compose instance for image transformations.
data_transforms = {
    'train': None,  
    'val': None,
}

### 3.1 Model Training
#### 3.1.1 Tasks
**(a).You are asked to complete the fish species classification task in three ways. Initialize a Resnet18 model using ALL of the following three ways:**

1. Using the pre-trained Resnet18 model as a fixed feature extractor (only fine-tune the fully-connected layer);
2. Fine-tuning all layers of the pre-trained Resnet18 model;
3. Training a Resnet18 model from scratch.

In [ ]:
# Replace 'None' with your dataset directory
train_dir = None
val_dir = None
test_dir = None

train_set = datasets.ImageFolder(train_dir,data_transforms['train'])
val_set = datasets.ImageFolder(val_dir, data_transforms['val'])
test_set = datasets.ImageFolder(test_dir, data_transforms['val'])

In [ ]:
num_classes = len(train_set.classes)
assert num_classes==4

In [ ]:
# TODO: Complete the following code
"""
1. Load a pre-trained ResNet-18 model as a model_extractor. (We have already done this for you)
2. Freeze all the layers of the model_extractor.
3. Replace the last fully connected layer (model_extractor.fc) with a new Linear layer that matches the number of classes.
4. Move the model to GPU if applicable.
"""

model_extractor = models.resnet18(pretrained=True)
# Your code starts here

In [ ]:
# TODO: Complete the following code
"""
1. Load a pre-trained ResNet-18 model as a model_finetune.
2. Replace the last fully connected layer (model_finetune.fc) with a new Linear layer that matches the number of classes.
3. Move the model to GPU if applicable.
"""

# Your code starts here
model_finetune = None

In [ ]:
# TODO: Complete the following code
"""
1. Load a random initialized ResNet-18 model as model_scratch.
2. Replace the last fully connected layer (model_scratch.fc) with a new Linear layer that matches the number of classes.
3. Move the model to GPU if applicable.
"""

# Your code starts here
model_scratch = None

**(b).Before training, can you speculate how the training will be different between thesethree approaches? Which way do you think is the most appropriate given this fishing dataset,and why?**

#### 3.1.2 Model Training/Fine-tuning 

Explain the design of your workflow. Your workflow should include the following components:
* Appropriate data transformation and augmentation;
* Loss function and optimizer;
* Training policy, including the number of epochs, batch size, learning rate. 

**Explain your workflow here**:



In [ ]:
# Replace 'None' with a torch.utils.data.DataLoader instance for image transformations.
train_loader = None
val_loader = None
test_loader = None

# Replace 'None' with a loss function. You might find useful information here https://pytorch.org/docs/stable/nn.html#loss-functions
criterion = None
# Replace 'None' with a optimizer. You might find useful information here https://pytorch.org/docs/stable/optim.html
optimizer = None
# Scheduler is not necessary, but you might find it could improve the model performance. Useful information can be found here https://pytorch.org/docs/stable/optim.html
scheduler = None

In [ ]:
def train(model, train_loader, optimizer, criterion, epoch, **kwargs):
    model.train()
    running_loss = 0.0
    # TODO: Complete this function based on the following instructions.

    for batch_idx, (data, target) in enumerate(train_loader):
        
        """
        1. Move the data and target tensors to the device (GPU preferable).
        2. Zero the gradients of the optimizer.
        3. Perform a forward pass through the model.
        4. Calculate the loss.
        5. Calculate the gradients.
        6. Update the model parameters using the optimizer.
        7. Add the loss for this batch to the running loss for the epoch.
        """

        # Your code here
        
    """
    1. You might want to print the loss every some iterations so you can monitor the training earlier.
    2. Calculate the epoch loss by dividing the running loss by the number of samples in the train_loader.
    3. Return the epoch loss.
    """
    # Your code here
    
    pass


In [ ]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            # TODO: Complete the following code
            """
            1. Move the data and target tensors to the device (GPU preferable).
            2. Perform a forward pass through the model to get the output.
            3. Calculate the test loss for this batch.
            4. Calculate the number of correct predictions for this batch.
            5. Accumulate the test loss and the number of correct predictions across all batches.
            """

            # Your code here
            
    # Calculate the average test loss and accuracy across all batches.
    # Return the average test loss.
    # Your code here
    
    pass

In [ ]:
total_epoch = 50
train_losses = []
val_losses = [] 
best_loss = 10000
best_model = None #You don't need to replace 'None' this line



for epoch in range(1, total_epoch + 1):
    # Train the model for one epoch and get the training loss
    train_loss = train(...) # Replace '...'
    train_losses.append(train_loss)
    
    # Validate the model on the validation set and get the validation loss
    val_loss = test(...) # Replace '...'
    val_losses.append(val_loss)

    # TODO: Complete the following code
    """
    1. Check if the current validation loss is lower than the best validation loss.
    2. If it is, update the best validation loss with the current validation loss.
    3. Also, update the best model parameters with the current model parameters.
    """

    # Your code here

    if scheduler is not None:
        scheduler.step()


### 3.2 Results
**(a) Generate a plot of training and validation loss curves for each of the three trainingcases. On the plots, mark the epoch that you selected as the best model,and explain your choice.**

In [ ]:
# Your code here

**(b) Choose a single best performing model as your final model, and report the confusion matrix, per-class accuracy, and weighted F1 score of this model on the validation set.**

To evaluate your model on the evaluation set, first use your best model to obtain predictions for the evaluation set.

In [ ]:
val_loss = test(...) # Replace '...'
print(val_loss)

true_label = []
pred_label = []
for data, target in val_loader:
    # Your code here

Then, show the performance of your best model on the evaluation set. You may find the scikit-learn library useful for this task, which provides several metrics for model evaluation. You can find more information on model evaluation metrics in scikit-learn here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
# Your code here

### 3.3 Model Performance Evaluation
Use your best performing model to predict the image labels for the held-out test set. Save your predictions in **`<YourNetID>`_CNN_predictions.csv** with the image filename (‘XXX.JPG’) in the first column, and your prediction (“ALB”, “YFT”, “OTHER”, OR “NoF”) in the second column.Please see Example_CNN_predictions.csv for the correct format.

In [ ]:
# Your code here

## Question 4 Generating Fake Images like Real
Your task is to generate images using Generative Adversarial Networks (GANs) based on the well-known MNIST dataset. The MNIST dataset consists of handwritten digits from 0 to 9 and is widely used in image classification applications. You may find some useful information in this PyTorch tutorial on DCGANs for generating images: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html. 

Please ensure you have access to a GPU for running this part of the project, as using a CPU may result in excessively long computation times. You can take advantage of the free GPU resources available on Google Colab.

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load MNIST data
transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
mnist_dataset = datasets.MNIST(root="./mnist_data", download=True, train=True, transform=transform)

# Replace 'None' with the appropriate values.
batch_size = None
data_loader = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 141047930.86it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 50536376.23it/s]


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 34998919.03it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 17287231.19it/s]


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



### 4.1 Design GAN's architecture

**(a) Design a Generator network to generate fake images.**

In [ ]:
image_size = 32

In [ ]:
class Generator(nn.Module):
    # TODO: Design the Generator architecture. 
    """

    1. Design a generator that takes a random input and outputs a tensor with the same shape as the real images.
    2. Consider using an nn.Sequential instance to wrap your layers. (self.main)
    3. Initializing the weights for the layers is optional but might improve training performance.
    """
    def __init__(self, latent_dim=None):
        super(Generator, self).__init__()
        self.main = None # Replace 'None' here
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            elif isinstance(m, (nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias.data, 0)

    def forward(self, input):
        return self.main(input)



**(b) Design a Discriminator network to distinguish real and fake images.**

In [ ]:
class Discriminator(nn.Module):
    # TODO: Design the Discriminator architecture. You only need to modify the self.main 
    """

    1. Design a discriminator that takes generator's outputs to discriminate whether the images is real images or fake images.
    2. Consider using an nn.Sequential instance to wrap your layers. (self.main)
    3. Initializing the weights for the layers is optional but might improve training performance.
    """

    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = None # Replace 'None' here
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            elif isinstance(m, (nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias.data, 0)
                
    def forward(self, input):
        return self.main(input)

### 4.2 Model Training

**(a) Initializing the networks, optimizers, and loss function.**

In [ ]:
# Replace 'None' with the appropriate values.
epochs = None
lr = None
generator = Generator().to(device)
discriminator = Discriminator().to(device)
criterion = None
optimizer_G = None
optimizer_D = None
criterion = None
noise = None
real_label = 1.
fake_label = 0.

**(b) Alternatively training the Discriminator and Generator.**



We have provided a portion of the starter code below to assist you in completing this task more easily. Please complete the provided code and ensure that it runs successfully.

In [ ]:
def train(train_loader, generator, discriminator, optimizer_G, optimizer_D, criterion):
    for i, (real_images, _) in enumerate(train_loader):
        # TODO: the code we provided below is incomplete, complete this function based on the following instructions. 

        """
        1. Move the real_images to the device (GPU preferable).
        
        Update discriminator D network: maximize log(D(x)) + log(1 - D(G(z)))
        2. Zero the gradients of the optimizer_D.    
        3. Get the discriminator's output for all real images.
        4. Calculate loss on all real images batch.
        5. Calculate gradients for D.
        6. Generate fake images/features using the generator G.
        7. Get the discriminator's output for all fake images.
        8. Calculate loss on all fake images batch.
        9. Calculate gradients for D, and accumulate with previous gradients.
        10. Update D.
        
        Update G network: maximize log(D(G(z)))
        11. Zero the gradients of the optimizer_G.
        12. Perform a forward pass of all fake images batch through the newly updated D.
        13. Calculate G's loss.
        14. Calculate gradients for G.
        15. Update G.
        
        16. Print the loss during the training to see whether the model is training well.
        17. Save the training losses. You'll be asked to visualize the G loss and D loss during the training.
        """
        
        #Update discriminator D network

        #Load the real images from dataloader, and then labeled them as real
        real_images = real_images.to(device)
        b_size = real_images.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        #your code here is to maximize log(D(x))


        #You will need a noise tensor to generate fake images, and then labeled them as fake
        noise = torch.randn(b_size, latent_dim, 1, 1, device=device)
        fake = generator(noise)
        label.fill_(fake_label)
        output = discriminator(fake.detach()).view(-1)
        #your code here is to maximize log(1 - D(G(z)))


        #Update G network
        #For this part, we provide a hint rather than specific code.
        #The goal of the generator is to create fake images that the discriminator classifies as real.
        #So, when updating the generator, label the fake images as real.
        #your code here is to maximize log(D(G(z)))

    pass


**(c) Plotting and saving the loss of both the Generator and Discriminator during training.**

In [ ]:
# Your code here

### 4.3 Evaluation

In [ ]:
#Save and upload the generator model you trained. 
student_id = None
torch.save(generator, f"{student_id}_GAN.pt")

If you want to visualize your generated images, you can use the following code:

In [ ]:
from torchvision.utils import save_image
from IPython.display import Image
#Typically, you will need random noise vectors to generate fake images. The generator you trained
#learns to map noise vectors to real images during the training process.
generated_images = generator(noise)
save_image(generated_images, "generated_samples.png",normalize=True)
Image(filename="generated_samples.png")